<a href="https://colab.research.google.com/github/Jaseem001/Text-to-Topics-and-Dates/blob/main/Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
import re
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import dateutil.parser as dparser
import nltk
import datetime as dt
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#@title
#function to convert number names to numbers
def text2int(textnum, numwords={}):
    if not numwords:
      units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
      ]
      tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]
      scales = ["hundred", "thousand", "million", "billion", "trillion"]
      numwords["and"] = (1, 0)
      for idx, word in enumerate(units):    numwords[word] = (1, idx)
      for idx, word in enumerate(tens):     numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):   numwords[word] = (10 ** (idx * 3 or 2), 0)
    current = result = 0
    for word in textnum.split():
        if word not in numwords:
          return word
        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0
    return result + current

In [ ]:
text = "Math test portions full module 1. Timing 9:30"
text = text.lower().split()
for i, word in enumerate(text):
    if text[i] != "and":
        text[i] = str(text2int(word))                                      #converting number names to integers
text = " ".join(text)
text = re.sub(r'(\d)\s+(\d)+', r'\1:\2', text)                            #converting space between time numbers to :

In [ ]:
#replacing the commonly occuring words with datetime values
text = text.replace("later", str(dt.datetime.now() + dt.timedelta(hours=2)))
text = text.replace(" coming ", " next ")  
text = text.replace("tonight", "8 pm") 
text = text.replace("morning", "8 am") 
text = text.replace("noon", " 12 pm") 
text = text.replace("evening", "4 pm")
text = text.replace("midnight", "12 am")
text = text.replace("this week", str(dt.datetime.now().date() + dt.timedelta((0-dt.datetime.now().weekday()) % 7 )))
text = text.replace("next week", str(dt.datetime.now().date() + dt.timedelta((0-dt.datetime.now().weekday()) % 7 )))
text = text.replace("next monday", str(dt.datetime.now().date() + dt.timedelta((0-dt.datetime.now().weekday()) % 7 )))
text = text.replace("next tuesday", str(dt.datetime.now().date() + dt.timedelta((1-dt.datetime.now().weekday()) % 7 )))
text = text.replace("next wednesday", str(dt.datetime.now().date() + dt.timedelta((2-dt.datetime.now().weekday()) % 7 )))
text = text.replace("next thursday", str(dt.datetime.now().date() + dt.timedelta((3-dt.datetime.now().weekday()) % 7 )))
text = text.replace("next friday", str(dt.datetime.now().date() + dt.timedelta((4-dt.datetime.now().weekday()) % 7 )))
text = text.replace("next saturday", str(dt.datetime.now().date() + dt.timedelta((5-dt.datetime.now().weekday()) % 7 )))
text = text.replace("next sunday", str(dt.datetime.now().date() + dt.timedelta((6-dt.datetime.now().weekday()) % 7 )))
text = text.replace("2 weeks", str(dt.datetime.now().date() + dt.timedelta(days=14))) 
text = text.replace("around this time", str(dt.datetime.now().strftime("%H:%M:%S"))) 
text = text.replace("this weekend", str(dt.datetime.now().date() + dt.timedelta((5-dt.datetime.now().date().weekday()) % 7 ))) 
text = text.replace("next weekend", str(dt.datetime.now().date() + dt.timedelta((5-dt.datetime.now().date().weekday()) % 7 ) + dt.timedelta(days=7)))
text = text.replace("weekend", str(dt.datetime.now().date() + dt.timedelta((5-dt.datetime.now().date().weekday()) % 7 ))) 
text = text.replace(" from today ", " ")
text = text.replace("day after tomorrow", str(dt.datetime.now().date() + dt.timedelta(days=2))) 
text = text.replace("day after", str(dt.datetime.now().date() + dt.timedelta(days=2))) 
text = text.replace("tomorrow", str(dt.datetime.now().date() + dt.timedelta(days=1))) 
text = text.replace("next day", str(dt.datetime.now().date() + dt.timedelta(days=1))) 
text = text.replace(" having ", " ")
text = text.replace("guys", "")
text = text.replace("guys", "")
text = text.replace("hey", "")
text = text.replace("hello", "")
text

'math test portions full module 1. timing 9:30'

In [ ]:
cal_date = []
parts = re.split(' to | - ', text)
for i, sentence in enumerate(parts):
    try:
        cal_date.append(dparser.parse(parts[i],fuzzy=True))
    except:
        pass
result = []
for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
        result.append(token)
reminder = " ".join(result).title()
reminder = re.sub('(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Sept|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)|Mon(?:day)?|Tue(?:sday)?|Wed(?:nesday)?|Thu(?:rsday)?|Fri(?:day)?|Sat(?:urday)?|Sun(?:day)?)', '', reminder)
print(reminder)
print(cal_date)

Math Test Portions Module Timing
[datetime.datetime(2020, 10, 1, 9, 30)]
